In [ ]:
%matplotlib qt
%gui qt5
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as op

from os.path import exists
import csv
import matplotlib.pyplot as plt
import mne

from mne.decoding import SlidingEstimator
from sklearn.utils import shuffle
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
import batch_average as ba
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# to save trained models and load using for making predictions
from joblib import dump, load
import scipy

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
import remove_diag as rd

## Decoding scores/matrices

In [ ]:
subject_models_df = pd.read_csv(r'D:\Master_thesis\Scores\Stereovsmono\subject_models_dfx {view}.csv')
print(subject_models_df)

# -------------------------- best model logic per subject and viewing type---------------------------

best_model_index = subject_models_df.\
    groupby(['subject_id', 'viewing_type'])['scores'].transform(max) == subject_models_df['scores']


best_model_replica = subject_models_df[best_model_index]
# subject_models_df[best_model_index]

best_model = best_model_replica.drop_duplicates(subset=['subject_id', 'viewing_type'], keep="first")
best_model

## Behavioural scores/matrices

In [ ]:
#importing trials results files for all subjects

overwrite = True
eyetracking_path = r'D:\Master_thesis\Final_data'
file_path = r'MainTask\Unity\S001\trial_results.csv'
                     
subjects_list = []
append_data = []

for subjects in os.listdir(eyetracking_path): 
    if subjects.startswith("VR2FEM_S"):
        df = pd.read_csv(os.path.join(eyetracking_path,subjects,file_path))
        append_data.append(df)
                           
eyetracking_data = pd.concat(append_data)

# print(eyetracking_data['emo'])   

In [ ]:
#plot confusion matrices

eyetracking_data_cf = eyetracking_data[eyetracking_data.selected_emotion != -1]

## Correlation between matrix

In [ ]:
rho_fisher_list = []
rho_fisher_list_mean = []
viewing_condition = ['mono', 'stereo']

for view in viewing_condition:
    rho_fisher_list = []
    for subjects in best_model['subject_id'].unique():
    
        prediction = best_model['predictions'].loc[(best_model['subject_id'] == subjects) & (best_model['viewing_type'] == view)]
        prediction_list = prediction.tolist()[0]
        p=prediction_list.split()
        p[0] = p[0].replace('[',"")
        p[-1] = p[-1].replace(']',"")
        p=[eval(i) for i in p]
        prediction_dm = p

        
        true= best_model['y_test'].loc[(best_model['subject_id'] == subjects) & (best_model['viewing_type'] == view)]
        true_list = true.tolist()[0]
        t=true_list.split()
        t[0] = t[0].replace('[',"")
        t[-1] = t[-1].replace(']',"")
        t=[eval(i) for i in t]
        true_dm = t
        if view=='mono': i=0 
        else: i=1
        prediction_bm = eyetracking_data_cf['selected_emotion'].loc[(eyetracking_data_cf['ppid'] == subjects) & (eyetracking_data_cf['stereo'] == i)]
        true_bm = eyetracking_data_cf['emo'].loc[(eyetracking_data_cf['ppid'] == subjects) & (eyetracking_data_cf['stereo'] == i)]

        cf_matrix_dm = confusion_matrix(true_dm, prediction_dm)
        cf_matrix_dm_nd = rd.remove_diag(cf_matrix_dm)


        cf_matrix_bm = confusion_matrix(true_bm, prediction_bm)
        cf_matrix_bm_nd = rd.remove_diag(cf_matrix_bm)
        if cf_matrix_bm.shape == (5,5):
            cf_matrix_bm_del = np.delete(cf_matrix_bm_nd, 4, 0)
            cf_matrix_bm_del = np.delete(cf_matrix_bm_del, 3, 1)

        cf_matrix_dm_spr = cf_matrix_dm_nd.flatten()
        cf_matrix_bm_spr = cf_matrix_bm_del.flatten()

        rho, pval = scipy.stats.spearmanr(cf_matrix_dm_spr,cf_matrix_bm_spr)

        rho_fisher = np.arctanh(rho)

        rho_fisher_list.append(rho_fisher)
    rho_fisher_mean= np.nanmean(rho_fisher_list)
    rho_fisher_list_mean.append(rho_fisher_mean)
print(rho_fisher_list_mean)